In [93]:
import pandas as pd
import random

In [94]:
word_counts = dict()
joints = {"neutral_pred": {}, "contradiction_pred": {}, "entailment_pred": {}, "neutral_actual": {}, "contradiction_actual": {}, "entailment_actual": {}}

In [95]:
def add_word(w, d):
    if w in d:
        d[w] += 1
    else:
        d[w] = 1

In [96]:
filename = "method1_eval/train_preds"
min_support = 20
min_conditional = 0.60
fracmin = "scaled"
fracmax = ""
preds = open(filename, 'r')
labels = open("../data/snli_1.0/cl_snli_train_lbl_file", 'r')
source = open("../data/snli_1.0/cl_snli_train_source_file", 'r')

In [97]:
num_samples = 0

try:
    while True:
        pred = next(preds).replace("\n", "")
        label = next(labels).replace("\n", "")
        hypothesis = set(next(source).lower().split("|||")[1].replace("\n", "").split(" "))
        for word in hypothesis:
            add_word(word, word_counts)
            add_word(word, joints[pred + "_pred"])
            add_word(word, joints[label + "_actual"])
        num_samples += 1
        if num_samples % 10000 == 0:
            print(num_samples, "completed")
except StopIteration:
    pass

10000 completed
20000 completed
30000 completed
40000 completed
50000 completed
60000 completed
70000 completed
80000 completed
90000 completed
100000 completed
110000 completed
120000 completed
130000 completed
140000 completed
150000 completed
160000 completed
170000 completed
180000 completed
190000 completed
200000 completed
210000 completed
220000 completed
230000 completed
240000 completed
250000 completed
260000 completed
270000 completed
280000 completed
290000 completed
300000 completed
310000 completed
320000 completed
330000 completed
340000 completed
350000 completed
360000 completed
370000 completed
380000 completed
390000 completed
400000 completed
410000 completed
420000 completed
430000 completed
440000 completed
450000 completed
460000 completed
470000 completed
480000 completed
490000 completed
500000 completed
510000 completed
520000 completed
530000 completed
540000 completed


In [98]:
words = pd.DataFrame.from_dict(word_counts, orient='index').rename(columns={0: "count"})
words.head()

,count
horse,2881
.,439149
a,329074
person,20025
his,29271


In [99]:
words = words.join(pd.DataFrame.from_dict(joints["neutral_pred"], orient='index').rename(columns={0: "neutral_pred"}))
words = words.join(pd.DataFrame.from_dict(joints["contradiction_pred"], orient='index').rename(columns={0: "contradiction_pred"}))
words = words.join(pd.DataFrame.from_dict(joints["entailment_pred"], orient='index').rename(columns={0: "entailment_pred"}))
words = words.join(pd.DataFrame.from_dict(joints["neutral_actual"], orient='index').rename(columns={0: "neutral_actual"}))
words = words.join(pd.DataFrame.from_dict(joints["contradiction_actual"], orient='index').rename(columns={0: "contradiction_actual"}))
words = words.join(pd.DataFrame.from_dict(joints["entailment_actual"], orient='index').rename(columns={0: "entailment_actual"}))
words.head()

,count,neutral_pred,contradiction_pred,entailment_pred,neutral_actual,contradiction_actual,entailment_actual
horse,2881,786.0,1087.0,1008.0,854.0,1096.0,931.0
.,439149,141398.0,144234.0,153517.0,145492.0,147546.0,146111.0
a,329074,106683.0,103194.0,119197.0,109939.0,106046.0,113089.0
person,20025,4544.0,4697.0,10784.0,4973.0,4852.0,10200.0
his,29271,14577.0,9103.0,5591.0,14343.0,9476.0,5452.0


In [100]:
words = words.sort_values("count", ascending=False).reset_index().rename(columns={"index": "word"})
words.head()

,word,count,neutral_pred,contradiction_pred,entailment_pred,neutral_actual,contradiction_actual,entailment_actual
0,.,439149,141398.0,144234.0,153517.0,145492.0,147546.0,146111.0
1,a,329074,106683.0,103194.0,119197.0,109939.0,106046.0,113089.0
2,the,242630,91442.0,89035.0,62153.0,91951.0,90757.0,59922.0
3,is,221473,71969.0,73989.0,75515.0,73973.0,75753.0,71747.0
4,are,125846,39797.0,39952.0,46097.0,41094.0,40852.0,43900.0


In [101]:
print(words.shape)
words = words[words["count"] > min_support]
words.shape

(31963, 8)


(5509, 8)

In [102]:
words.head()

,word,count,neutral_pred,contradiction_pred,entailment_pred,neutral_actual,contradiction_actual,entailment_actual
0,.,439149,141398.0,144234.0,153517.0,145492.0,147546.0,146111.0
1,a,329074,106683.0,103194.0,119197.0,109939.0,106046.0,113089.0
2,the,242630,91442.0,89035.0,62153.0,91951.0,90757.0,59922.0
3,is,221473,71969.0,73989.0,75515.0,73973.0,75753.0,71747.0
4,are,125846,39797.0,39952.0,46097.0,41094.0,40852.0,43900.0


In [103]:
words["neutral_pred"] = words["neutral_pred"] / words["count"]
words["contradiction_pred"] = words["contradiction_pred"] / words["count"]
words["entailment_pred"] = words["entailment_pred"] / words["count"]
words["neutral_actual"] = words["neutral_actual"] / words["count"]
words["contradiction_actual"] = words["contradiction_actual"] / words["count"]
words["entailment_actual"] = words["entailment_actual"] / words["count"]
words.head()

,word,count,neutral_pred,contradiction_pred,entailment_pred,neutral_actual,contradiction_actual,entailment_actual
0,.,439149,0.321982,0.328440,0.349578,0.331304,0.335982,0.332714
1,a,329074,0.324192,0.313589,0.362219,0.334086,0.322256,0.343658
2,the,242630,0.376878,0.366958,0.256164,0.378976,0.374055,0.246969
3,is,221473,0.324956,0.334077,0.340967,0.334005,0.342042,0.323954
4,are,125846,0.316236,0.317467,0.366297,0.326542,0.324619,0.348839


In [104]:
words = words[(words["neutral_pred"] > min_conditional) | (words["contradiction_pred"] > min_conditional) | (words["entailment_pred"] > min_conditional)]
print(words.shape)
words.head()

(1663, 8)


,word,count,neutral_pred,contradiction_pred,entailment_pred,neutral_actual,contradiction_actual,entailment_actual
11,to,46667,0.620288,0.202541,0.177170,0.594660,0.221934,0.183406
21,for,24764,0.726256,0.155145,0.118600,0.694597,0.177516,0.127887
22,there,24466,0.149677,0.216014,0.634309,0.179555,0.223044,0.597400
25,outside,22320,0.187007,0.115457,0.697536,0.219176,0.124910,0.655914
58,near,7129,0.184738,0.144761,0.670501,0.214616,0.153037,0.632347


In [105]:
words = words.fillna(0)

In [106]:
words.loc[words["neutral_pred"] > min_conditional, "highest_pred"] = "neutral"
words.loc[words["contradiction_pred"] > min_conditional, "highest_pred"] = "contradiction"
words.loc[words["entailment_pred"] > min_conditional, "highest_pred"] = "entailment"
words.loc[(words["neutral_actual"] > words["contradiction_actual"]) & (words["neutral_actual"] > words["entailment_actual"]), "highest_label"] = "neutral"
words.loc[(words["contradiction_actual"] > words["neutral_actual"]) & (words["contradiction_actual"] > words["entailment_actual"]), "highest_label"] = "contradiction"
words.loc[(words["entailment_actual"] > words["contradiction_actual"]) & (words["entailment_actual"] > words["neutral_actual"]), "highest_label"] = "entailment"
words.head()

,word,count,neutral_pred,contradiction_pred,entailment_pred,neutral_actual,contradiction_actual,entailment_actual,highest_pred,highest_label
11,to,46667,0.620288,0.202541,0.177170,0.594660,0.221934,0.183406,neutral,neutral
21,for,24764,0.726256,0.155145,0.118600,0.694597,0.177516,0.127887,neutral,neutral
22,there,24466,0.149677,0.216014,0.634309,0.179555,0.223044,0.597400,entailment,entailment
25,outside,22320,0.187007,0.115457,0.697536,0.219176,0.124910,0.655914,entailment,entailment
58,near,7129,0.184738,0.144761,0.670501,0.214616,0.153037,0.632347,entailment,entailment


In [107]:
words.tail()

,word,count,neutral_pred,contradiction_pred,entailment_pred,neutral_actual,contradiction_actual,entailment_actual,highest_pred,highest_label
5501,became,21,0.714286,0.238095,0.047619,0.714286,0.238095,0.047619,neutral,neutral
5503,boiling,21,0.190476,0.666667,0.142857,0.190476,0.619048,0.190476,contradiction,contradiction
5505,jealous,21,0.904762,0.000000,0.095238,0.857143,0.047619,0.095238,neutral,neutral
5507,largest,21,0.761905,0.238095,0.000000,0.666667,0.333333,0.000000,neutral,neutral
5508,inning,21,0.857143,0.142857,0.000000,0.809524,0.190476,0.000000,neutral,neutral


In [108]:
words["highest_pred"].value_counts()

neutral          1074
contradiction     466
entailment        123
Name: highest_pred, dtype: int64

In [109]:
words[words["highest_pred"] != words["highest_label"]]

,word,count,neutral_pred,contradiction_pred,entailment_pred,neutral_actual,contradiction_actual,entailment_actual,highest_pred,highest_label
5214,fleeing,23,0.652174,0.347826,0.000000,0.434783,0.521739,0.043478,neutral,contradiction
5369,joking,21,0.666667,0.047619,0.285714,0.380952,0.190476,0.428571,neutral,entailment


In [110]:
# def computefrac(minProb, minfrac, maxfrac, prob):
#     probrange = 1 - minProb
#     probperc = (prob - minProb) / probrange
#     fracrange = maxfrac - minfrac
#     return (fracrange * probperc) + minfrac

def computefrac(inProb, minfrac, maxfrac, prob):
    prob_reduce = prob - ((1 - prob) / 2)
    return prob_reduce / prob

In [111]:
labels = open("../data/snli_1.0/cl_snli_train_lbl_file", 'r')
source = open("../data/snli_1.0/cl_snli_train_source_file", 'r')
output = open("min%s,%s-%s-samples.txt" % (min_conditional, fracmin, fracmax), 'w')
output_labels = open("min%s,%s-%s-samples_labels.txt" % (min_conditional, fracmin, fracmax), 'w')

In [112]:
import random

num_samples = 0
dropped_samples = 0

random.seed(42)

new_source = []
new_labels = []

try:
    while True:
        label = next(labels).replace("\n", "")
        sample = next(source)
        sample = sample.split("|||")
        premise, hypothesis = sample[0], sample[1]
        new_source.append(premise + "|||")
        sample_prob = -1
        for word in hypothesis.split(" "):
            subset = words[(words["word"] == word.lower()) & (words["highest_label"] == label)]
            if subset.shape[0] and subset[label + "_actual"].iloc[0] > min_conditional:
                prob = computefrac(min_conditional, fracmin, fracmax, subset[label + "_actual"].iloc[0])
                sample_prob = max(sample_prob, prob)
        num = random.random()
        if num > sample_prob:
            new_source.append(hypothesis)
            new_labels.append(label + "\n")
        else:
            dropped_samples += 1
        num_samples += 1
        # if num_samples == 3:
        #     print(repr("".join(new_source)))
        #     break
        if num_samples % 10000 == 0:
            print(num_samples, "completed")
            print(dropped_samples, "dropped")
except StopIteration:
    pass

print(dropped_samples, "dropped")
new_source = "".join(new_source)
new_labels = "".join(new_labels)

10000 completed
2588 dropped
20000 completed
5165 dropped
30000 completed
7786 dropped
40000 completed
10475 dropped
50000 completed
13165 dropped
60000 completed
15822 dropped
70000 completed
18495 dropped
80000 completed
21048 dropped
90000 completed
23665 dropped
100000 completed
26230 dropped
110000 completed
28905 dropped
120000 completed
31598 dropped
130000 completed
34252 dropped
140000 completed
36915 dropped
150000 completed
39570 dropped
160000 completed
42200 dropped
170000 completed
44804 dropped
180000 completed
47423 dropped
190000 completed
50048 dropped
200000 completed
52715 dropped
210000 completed
55360 dropped
220000 completed
58019 dropped
230000 completed
60627 dropped
240000 completed
63266 dropped
250000 completed
65871 dropped
260000 completed
68458 dropped
270000 completed
71078 dropped
280000 completed
73785 dropped
290000 completed
76386 dropped
300000 completed
79081 dropped
310000 completed
81743 dropped
320000 completed
84443 dropped
330000 completed
870

In [113]:
output.write(new_source)
output_labels.write(new_labels)

4508549

In [114]:
word_counts = dict()
joints = {"neutral_new": {}, "contradiction_new": {}, "entailment_new": {}}

labels = open("../data/snli_1.0/cl_snli_train_lbl_file", 'r')
source = open("min%s,%s-%s-fixed.txt" % (min_conditional, fracmin, fracmax), 'r')

In [115]:
num_samples = 0

try:
    while True:
        pred = next(source).replace("\n", "")
        label = next(labels).replace("\n", "")
        hypothesis = set(next(source).lower().split("|||")[1].replace("\n", "").split(" "))
        for word in hypothesis:
            add_word(word, word_counts)
            add_word(word, joints[label + "_new"])
        num_samples += 1
        if num_samples % 10000 == 0:
            print(num_samples, "completed")
except StopIteration:
    pass

10000 completed
20000 completed
30000 completed
40000 completed
50000 completed
60000 completed
70000 completed
80000 completed
90000 completed
100000 completed
110000 completed
120000 completed
130000 completed
140000 completed
150000 completed
160000 completed
170000 completed
180000 completed
190000 completed
200000 completed
210000 completed
220000 completed
230000 completed
240000 completed
250000 completed
260000 completed
270000 completed


In [116]:
new_words = pd.DataFrame.from_dict(word_counts, orient='index').rename(columns={0: "count_new"})
new_words.head()

,count_new
.,219452
a,164762
an,11252
person,9974
at,20215


In [117]:
new_words = new_words.join(pd.DataFrame.from_dict(joints["neutral_new"], orient='index').rename(columns={0: "neutral_new"}))
new_words = new_words.join(pd.DataFrame.from_dict(joints["contradiction_new"], orient='index').rename(columns={0: "contradiction_new"}))
new_words = new_words.join(pd.DataFrame.from_dict(joints["entailment_new"], orient='index').rename(columns={0: "entailment_new"}))
new_words.head()

,count_new,neutral_new,contradiction_new,entailment_new
.,219452,73123.0,73282.0,73047.0
a,164762,54893.0,54917.0,54952.0
an,11252,3719.0,3784.0,3749.0
person,9974,3287.0,3402.0,3285.0
at,20215,6726.0,6685.0,6804.0


In [118]:
new_words = new_words.sort_values("count_new", ascending=False).reset_index().rename(columns={"index": "word"})
new_words.head()

,word,count_new,neutral_new,contradiction_new,entailment_new
0,.,219452,73123.0,73282.0,73047.0
1,a,164762,54893.0,54917.0,54952.0
2,the,121096,40266.0,40421.0,40409.0
3,is,110732,36886.0,36886.0,36960.0
4,are,62794,20894.0,20974.0,20926.0


In [119]:
new_words["neutral_new"] = new_words["neutral_new"] / new_words["count_new"]
new_words["contradiction_new"] = new_words["contradiction_new"] / new_words["count_new"]
new_words["entailment_new"] = new_words["entailment_new"] / new_words["count_new"]
new_words.head()

,word,count_new,neutral_new,contradiction_new,entailment_new
0,.,219452,0.333207,0.333932,0.332861
1,a,164762,0.333165,0.333311,0.333524
2,the,121096,0.332513,0.333793,0.333694
3,is,110732,0.333111,0.333111,0.333779
4,are,62794,0.332739,0.334013,0.333248


In [120]:
combined = words.merge(new_words, how='left', on="word")
combined.head()

,word,count,neutral_pred,contradiction_pred,entailment_pred,neutral_actual,contradiction_actual,entailment_actual,highest_pred,highest_label,count_new,neutral_new,contradiction_new,entailment_new
0,to,46667,0.620288,0.202541,0.177170,0.594660,0.221934,0.183406,neutral,neutral,23543.0,0.333900,0.334622,0.331479
1,for,24764,0.726256,0.155145,0.118600,0.694597,0.177516,0.127887,neutral,neutral,5823.0,0.337455,0.336596,0.325949
2,there,24466,0.149677,0.216014,0.634309,0.179555,0.223044,0.597400,entailment,entailment,12166.0,0.322867,0.335279,0.341854
3,outside,22320,0.187007,0.115457,0.697536,0.219176,0.124910,0.655914,entailment,entailment,6343.0,0.337064,0.337695,0.325240
4,near,7129,0.184738,0.144761,0.670501,0.214616,0.153037,0.632347,entailment,entailment,1976.0,0.327429,0.339575,0.332996


In [121]:
combined.shape

(1663, 14)

In [122]:
(combined["count"] - combined["count_new"]).sum()

335372.0